In [ ]:
import pandas as pd
rway_img_df_ok = pd.read_csv("collection_images_download.csv")
rway_img_df = rway_img_df_ok.copy()

# EXTRACT BRAND FROM IMAGE NAME
# Creazione delle nuove colonne
rway_img_df['brand'] = rway_img_df['image_name'].str.lower()
rway_img_df['brand'] = rway_img_df['brand'].str.split('couture').str[0]
rway_img_df['brand'] = rway_img_df['brand'].str.split('ready-to-wear').str[0]
rway_img_df['brand'] = rway_img_df['brand'].str.split('menswear').str[0]
rway_img_df['brand'] = rway_img_df['brand'].str.replace("-", " ")
rway_img_df['brand'] = rway_img_df['brand'].str.strip()

# subset rway_img_df where brand cointains versage,hermes
#brand_keywords = ['rick owens', 'louis vuitton', 'versace', 'giorgio armani', 'hermes', 'dries van noten', 'chanel']
brand_keywords = ['rick owens', 'dries van noten','versace']

data = rway_img_df[rway_img_df['brand'].isin(brand_keywords)]

# rename brand in target column
data['target'] = data['brand']
# factorized 'brand' using pandas
data['target'] = pd.factorize(data['target'])[0]


In [ ]:
# Calculate the value counts of each brand in the 'brand' column
brand_counts = rway_img_df['brand'].value_counts()

# Print the brand names with count greater than 800
for brand, count in brand_counts.items():
    if count > 800:
        print(brand, count)

#! here the list of brand to predict without any class imbalance
#* ['rick owens', 'yohji yamamoto', 'louis vuitton', 'versace', 'giorgio armani', 'hermes', 'dries van noten', 'chanel']


In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input
from tensorflow.keras.models import Model

# My custom simple_vgg16 
def simple_vgg16(input_shape=(224, 224, 3), num_classes=10):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=input_shape))
    model.add(tf.keras.layers.MaxPooling2D((2, 2)))
    model.add(tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(tf.keras.layers.MaxPooling2D((2, 2)))
    model.add(tf.keras.layers.Dense(256, activation='relu'))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(256, activation='relu'))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(num_classes, activation='softmax'))
    return model

# Load the image data and labels

# Split the data into training and validation sets
X_train_paths, X_val_paths, y_train, y_val = train_test_split(
    data['image_path'],
    data['target'],
    test_size=0.2,
    random_state=1234
)

# Define image dimensions (img_height and img_width) to resize the images during preprocessing.
img_height, img_width = 224, 224

# Map the brand labels in y_train and y_val to their corresponding integer labels using the brand_to_label dictionary.
brand_to_label = {brand: i for i, brand in enumerate(np.unique(data['target']))}

# Map brand labels to integer labels
y_train = np.array([brand_to_label[brand] for brand in y_train])
y_val = np.array([brand_to_label[brand] for brand in y_val])

# Calculate the number of classes (num_classes) based on the unique labels.
num_classes = len(brand_to_label)

# Define the preprocess_image function to load, resize, and normalize the pixel values of the images.
def preprocess_image(image_path):
    image_path = './imgs/images/images/' + image_path  # Add the path prefix
    image = load_img(image_path, target_size=(img_height, img_width))
    image = img_to_array(image)
    image = image / 255.0  # Normalize pixel values between 0 and 1
    return image

# Preprocess the images in X_train and X_val by applying the preprocess_image function to each image path.
X_train = np.array([preprocess_image(path) for path in X_train_paths])
X_val = np.array([preprocess_image(path) for path in X_val_paths])

# Convert the integer labels in y_train and y_val to one-hot encoded vectors using tf.keras.utils.to_categorical.
y_train = tf.keras.utils.to_categorical(y_train, num_classes=num_classes)
y_val = tf.keras.utils.to_categorical(y_val, num_classes=num_classes)

# Create the custom simple_vgg16 model
input_shape = (img_height, img_width, 3)
model = simple_vgg16(input_shape=input_shape, num_classes=num_classes)

# Compile the model with an appropriate optimizer and loss function (e.g., 'adam' optimizer and 'categorical_crossentropy' loss)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model using the preprocessed data
history_100_new2 = model.fit(X_train, y_train, batch_size=32, epochs=10, validation_data=(X_val, y_val))
